#### Get latest data

In [ ]:
import requests
import pandas as pd
import json
import os
from datetime import datetime, timedelta

In [ ]:
import requests

url = "https://api.coingecko.com/api/v3/coins/bitcoin/ohlc"

querystring = {"vs_currency":"usd","days":"14","precision":"full"}

headers = {"x-cg-demo-api-key": ""}

response = requests.get(url, headers=headers, params=querystring)
    
print(response.json())

In [ ]:
result = pd.DataFrame(response.json(), columns=['timestamp', 'open', 'high', 'low', 'close'])
result['timestamp'] = pd.to_datetime(result['timestamp'], unit='ms')
result

In [ ]:
import requests
import pandas as pd
import os

API_KEY = os.getenv("COINGECKO_API_KEY", "")

def get_ohlc_data(coin_id, vs_currency="usd", days=1, precision="full"):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/ohlc"
    querystring = {"vs_currency": vs_currency, "days": days, "precision": precision}
    headers = {"x-cg-demo-api-key": API_KEY} if API_KEY else {}
    
    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = pd.DataFrame(response.json(), columns=['timestamp', 'open', 'high', 'low', 'close'])
        data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

In [ ]:
data = get_ohlc_data(coin_id='bitcoin', vs_currency='usd', days=14, precision='full')

In [ ]:
data.to_csv("/home/anhtt1/Workspace/DE/Project/DE_Project/projects/trade_template/data/bitcoin_ohlc.csv", index=False)

In [ ]:
df = pd.read_csv('/home/anhtt1/Workspace/DE/Project/DE_Project/projects/trade_template/data/bitcoin_ohlc.csv')

In [ ]:
# --- EMA ---
df["EMA_20"] = df["close"].ewm(span=20, adjust=False).mean()
df["EMA_50"] = df["close"].ewm(span=50, adjust=False).mean()

In [ ]:
# --- RSI ---
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df["RSI_14"] = compute_rsi(df["close"], 14)

In [ ]:
df.to_csv("/home/anhtt1/Workspace/DE/Project/DE_Project/projects/trade_template/data/bitcoin_ohlc_with_indicators.csv", index=False)

#### Incremental

In [ ]:
incremental_data = data = get_ohlc_data(coin_id='bitcoin', vs_currency='usd', days=14, precision='full')

In [ ]:
incremental_data

In [ ]:
# --- EMA ---
incremental_data["EMA_20"] = incremental_data["close"].ewm(span=20, adjust=False).mean()
incremental_data["EMA_50"] = incremental_data["close"].ewm(span=50, adjust=False).mean()
incremental_data["RSI_14"] = compute_rsi(incremental_data["close"], 14)

In [ ]:
incremental_data

In [ ]:
df = pd.read_csv('/home/anhtt1/Workspace/DE/Project/DE_Project/projects/trade_template/data/bitcoin_ohlc_with_indicators.csv')

In [ ]:
import pandas as pd

# Convert timestamp về datetime cho chắc chắn
df["timestamp"] = pd.to_datetime(df["timestamp"])
incremental_data["timestamp"] = pd.to_datetime(incremental_data["timestamp"])


In [ ]:
# Tìm những timestamp có ở df2 nhưng không có ở df1
missing_timestamps = incremental_data.loc[~incremental_data["timestamp"].isin(df["timestamp"])]


In [ ]:
missing_timestamps

In [ ]:
df_updated = pd.concat([df, missing_timestamps], ignore_index=True)
df_updated = df_updated.sort_values("timestamp").reset_index(drop=True)


In [ ]:
df_updated.to_csv("/home/anhtt1/Workspace/DE/Project/DE_Project/projects/trade_template/data/bitcoin_ohlc_with_indicators.csv", index=False)

#### Dash

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(df["close"], label="Giá đóng cửa", color="black")
plt.plot(df["EMA_20"], label="EMA 20", color="blue", linestyle="--")
plt.plot(df["EMA_50"], label="EMA 50", color="red", linestyle="--")
plt.title("Biểu đồ giá + EMA")
plt.xlabel("Thời gian (4h mỗi điểm)")
plt.ylabel("Giá")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import plotly.graph_objs as go
from dash import Dash, dcc, html

# Tạo Dash app
app = Dash(__name__)

# Biểu đồ giá + EMA
price_chart = go.Figure()
price_chart.add_trace(go.Scatter(y=df["close"], mode="lines", name="Giá đóng cửa", line=dict(color="black")))
price_chart.add_trace(go.Scatter(y=df["EMA_20"], mode="lines", name="EMA 20", line=dict(color="blue", dash="dash")))
price_chart.add_trace(go.Scatter(y=df["EMA_50"], mode="lines", name="EMA 50", line=dict(color="red", dash="dash")))
price_chart.update_layout(title="Biểu đồ Giá + EMA", height=400)

# Biểu đồ RSI
rsi_chart = go.Figure()
rsi_chart.add_trace(go.Scatter(y=df["RSI_14"], mode="lines", name="RSI 14", line=dict(color="purple")))
rsi_chart.add_trace(go.Scatter(y=[70]*len(df), mode="lines", name="Overbought", line=dict(color="red", dash="dot")))
rsi_chart.add_trace(go.Scatter(y=[30]*len(df), mode="lines", name="Oversold", line=dict(color="green", dash="dot")))
rsi_chart.update_layout(title="Biểu đồ RSI", height=300)

# Layout Dash
app.layout = html.Div([
    html.H1("Dashboard EMA & RSI", style={"textAlign": "center"}),
    dcc.Graph(figure=price_chart),
    dcc.Graph(figure=rsi_chart)
])

# Chạy Dash trong notebook
# Nếu muốn hiển thị trực tiếp trong notebook:
app.run(mode="inline")

# Nếu muốn mở ra tab trình duyệt riêng:
app.run(port=8050)

# if __name__ == "__main__":
#     app.run_server(debug=True)